In [1]:
import bitarray
from hashlib import sha256
from functools import partial
import random
import math
import string

random.seed(23)

# Bloom Filter

- you can ..
    - add items
    - check if a item is maybe contained in the set (in other words: it can answer the question "Is this item contained?" only with "maybe" or "no")
- you can not ..
    - check which items are in the set
- main advantage: constant space
- parameter
    - bit count
    - hash function count
- use cases
    - pre-cache for browser -> to check if browser cache contains the item, so you only check if it's likely in the cache (slow operation because of hard drive)
    - pre-cache for db -> to check if a row that contains the value the users asks for exist, and only if the answer is maybe, evaluate the query (so this speeds up querys that would otherwise return an empty set)
    - track posts shown to user -> so you know which post the user definitely hasn't seen yet, and you can show new posts

# Implementation of a simple Bloom Filter

In [2]:
class Bloomfilter:
    
    def __init__(self, bit_count, hashfunction_count):
        self.bit_count = bit_count
        self.hashfunction_count = hashfunction_count
        self.bits = [0] * self.bit_count
        self.salts = [str(x) for x in range(self.hashfunction_count)]
        self.hash_functions = [partial(self._base_hash_function, salt=salt) for salt in self.salts]
        
    def _base_hash_function(self, item, salt):
        return int(sha256(item.encode() + salt.encode()).hexdigest(), 16) % self.bit_count
    
    def add(self, item):
        for hash_function in self.hash_functions:
            index = hash_function(item)
            self.bits[index] = 1
    
    def is_maybe_present(self, item):
        for hash_function in self.hash_functions:
            index = hash_function(item)
            if self.bits[index] == 0:
                return False
        return True

# Test the implementation

#### Create Bloom Filter with 5 bits and 2 hash functions

In [3]:
bloom = Bloomfilter(5, 2)

#### Add the item "abc"

In [4]:
bloom.add("abc")

#### Verify that the set says that "abc" is maybe present

In [5]:
bloom.is_maybe_present("abc")

True

#### Check what the set says about other items

In [6]:
bloom.is_maybe_present("xyz")

False

In [7]:
chars = "abcdefghijklmnopqrstuvwxyz"

for c in chars:
    print(f"Set perhaps contains \"{c}\": {bloom.is_maybe_present(c)}")

Set perhaps contains "a": False
Set perhaps contains "b": False
Set perhaps contains "c": False
Set perhaps contains "d": False
Set perhaps contains "e": True
Set perhaps contains "f": False
Set perhaps contains "g": False
Set perhaps contains "h": True
Set perhaps contains "i": False
Set perhaps contains "j": False
Set perhaps contains "k": False
Set perhaps contains "l": False
Set perhaps contains "m": False
Set perhaps contains "n": False
Set perhaps contains "o": False
Set perhaps contains "p": True
Set perhaps contains "q": False
Set perhaps contains "r": True
Set perhaps contains "s": False
Set perhaps contains "t": False
Set perhaps contains "u": False
Set perhaps contains "v": True
Set perhaps contains "w": False
Set perhaps contains "x": False
Set perhaps contains "y": False
Set perhaps contains "z": False


# How to set the parameters

- $\textrm{bit count} = - \cfrac{n \cdot \ln \epsilon}{(\ln 2)^2}$
- $\textrm{hashfunction count} = - \log_2 \epsilon$
- with:
    - $n$ = anticipated number of unique items
    - $\epsilon$ = desired FP-rate

In [8]:
def estimate_bit_count(expected_unique_item_count, desired_fp_rate):
    return round(- (expected_unique_item_count * math.log(desired_fp_rate)) / (math.log(2)) ** 2)


def estimate_hashfunction_count(desired_fp_rate):
    return round(- math.log2(desired_fp_rate))

### Example

Assuming we expect to insert 5000 unique items ($n = 5000)$ and aim for a FP-rate of 1% ($\epsilon = 0.01$).

In [9]:
expected_unique_item_count = 5000
desired_fp_rate = 0.01

estimate_bit_count(expected_unique_item_count, desired_fp_rate), estimate_hashfunction_count(desired_fp_rate)

(47925, 7)

### Verify that this configuration results in the desired FP rate for a test run

In [12]:
bloom = Bloomfilter(47925, 7)


for _ in range(5000):
    random_string = "".join(random.choices(string.ascii_lowercase, k=20))
    bloom.add(random_string)
    
    
def false_positive_rate(bloomfilter, test_size):
    fp_count = 0
    for _ in range(test_size):
        random_string = "".join(random.choices(string.ascii_lowercase, k=20))
        if bloom.is_maybe_present(random_string):
            fp_count += 1
    fp_rate = fp_count / test_size
    return fp_rate


fp_rate = false_positive_rate(bloom, test_size=100000)
fp_rate

0.00975

### What happens if we underestimated the number of unique items and add double the estimated amount?

In [13]:
bloom = Bloomfilter(47925, 7)


for _ in range(10000):
    random_string = "".join(random.choices(string.ascii_lowercase, k=20))
    bloom.add(random_string)
    
    
fp_rate = false_positive_rate(bloom, test_size=100000)
fp_rate

0.15772

-> so instead of a fp error rate of 1%, we get a fp error rate of 15%!